<h1 style="text-align: center;">FEM on unbounded domains</h1>

This notebook shows how to run finite element computations on unbounded domains with *femtoscope*.

*prerequisites* :
- FEM knowledge
- Mesh Generation notebook
- FEM bounded notebook

The following code is adapted from scripts provided in the same directory as this notebook.

**If you have questions/comments or want to report a bug, please send me an email at <a href="mailto:hugo.levy@onera.fr">hugo.levy@onera.fr</a>**

In [1]:
# Add femtoscope to the path
%reset
%matplotlib inline
import sys
sys.path.append("../") # go to parent dir

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## Techniques to deal with asymptotic boundary conditions

Here we illustrate three techniques to deal with asymptotic boundary condition (i.e. when the value of the unknown is only at spatial infinity, assuming that is asymptotically isotropic). These techniques are the one described in [1], namely:
- Compactification of the unbounded domain $\Omega \mapsto \tilde{\Omega}$ (`PoissonCompact` and `ChameleonCompact` classes)
- Techniques based on domain splitting $\bar{\Omega} = \bar{\Omega}_{\mathrm{int}} \cup \bar{\Omega}_{\mathrm{ext}}$ and kelvin-inversion of the exterior domain $\Omega_{\mathrm{ext}} \mapsto \tilde{\Omega}_{\mathrm{ext}}$ (`PoissonSplit` and `ChameleonSplit` classes)
    - Virtual connection of DOFs
    - Ping-pong
    
These techniques are illustrated on the Poisson equation. More precisely, we wish to solve the following problem:
$$
\begin{cases} 
  \Delta u = \alpha \rho \text{ in } \Omega\\
  u(\mathbf{x}) \underset{\|\mathbf{x}\| \to +\infty}{\longrightarrow} 0
\end{cases} \, ,
$$
In order to assess the accuracy of the various implementations, we can compare the numerical approximation to the exact solution in the particular case of a perfect sphere (for which the Newtonian potential is known).
    
[1] https://arxiv.org/abs/2209.07226

In [2]:
from femtoscope.physics.poisson import PoissonCompact, PoissonSplit
from femtoscope.inout.meshfactory import MeshingTools, mesh_from_geo
from femtoscope.misc.analytical import potential_sphere
import numpy as np
from numpy import sqrt, pi, cos, sin, arccos

# Common parameters
coorsys = 'polar'
fem_order = 2
Rcut = 5.0
R_ball = 1.0
alpha = 4*pi
rho = 1.0

# Random sampling & analytical solution
np.random.seed(503)
X_rand = np.random.uniform(-Rcut, Rcut, 10000)
np.random.seed(1165)
Y_rand = np.random.uniform(-Rcut, Rcut, 10000)
coors_cart_rand = np.concatenate((X_rand[:, np.newaxis], Y_rand[:, np.newaxis]), axis=1)
coors_cart_rand = coors_cart_rand[np.where(np.linalg.norm(coors_cart_rand, axis=1)<0.95*Rcut)]
X_rand, Y_rand = coors_cart_rand[:, 0], coors_cart_rand[:, 1]
coors_cart_2d = np.ascontiguousarray(coors_cart_rand)
rr = np.linalg.norm(coors_cart_2d, axis=1)
sol_ana = potential_sphere(rr, R_ball, 1.0, rho=rho).squeeze()

### (i) Compactification

In [3]:
from copy import deepcopy

def rho_func(coors):
    value = np.zeros(coors.shape[0])
    norm2 = coors[:, 0]**2
    theta = coors[:, 1]
    boolin = np.where(norm2*((sin(theta)/R_ball)**2+(cos(theta)/R_ball)**2)<1)[0]
    value[boolin] = rho
    return value
    
def rho_func_mod(coors):
    """We apply the compactification transform on the density function."""
    coors_mod = deepcopy(coors)
    coors_mod[:, 0] = coors_mod[:, 0] / (Rcut - coors_mod[:, 0])
    return rho_func(coors_mod)

meshfile = "mesh_theta_compact.vtk"
try:
    mesh_from_geo('mesh_theta_compact.geo', show_mesh=True,
                  param_dic={'size' : 0.1, 'sa' : R_ball, 'Rc' : Rcut})
except FileNotFoundError:
    print("femtoscope\data\mesh\geo\mesh_theta_compact.geo cannot be found!")
    MT = MeshingTools(dimension=2)
    MT.create_rectangle(dx=Rcut, dy=pi, centered=False)
    MT.create_subdomain(CellSizeMin=0.05, CellSizeMax=0.05, DistMax=0.0)
    MT.generate_mesh(meshfile, show_mesh=True, unique_boundary=False,
                     ignoreTags=[200, 202, 203])

pb_compact = PoissonCompact(alpha, rho_func_mod, meshfile, coorsys=coorsys, fem_order=fem_order)
pb_compact.solve()
sol_compact = pb_compact.solver.sols[0]
field_compact = pb_compact.solver.weakforms[0].field
theta = arccos(Y_rand/rr)
eta = Rcut*rr/(1+rr)
coors_eval = np.ascontiguousarray(np.concatenate((eta[:, np.newaxis], theta[:, np.newaxis]), axis=1))
fem_compact = field_compact.evaluate_at(coors_eval, sol_compact[:, np.newaxis]).squeeze()
print("#DOFs = {}".format(sol_compact.shape[0]))
print("mean pointwise relative error: {:.2E}".format(np.mean(abs((fem_compact-sol_ana)/sol_ana))))

#DOFs = 7741
mean pointwise relative error: 1.19E-06


### (ii) Virtual connection of DOFs

In [4]:
conn = 'connected' # use the virtual connection of DOFs technique

meshfile_int = "mesh_theta_int.vtk"
meshfile_ext = "mesh_theta_ext.vtk"

try:
    size = 0.1
    mesh_from_geo('mesh_theta_int.geo', show_mesh=True,
                  param_dic={'size' : size, 'Ngamma' : int(1.5*pi/size), 'Rc' : Rcut, 'sa' : R_ball})
    mesh_from_geo('mesh_theta_ext.geo', show_mesh=True,
                  param_dic={'size' : size, 'Ngamma' : int(1.5*pi/size), 'Rc' : Rcut})
    meshfiles = [meshfile_int, meshfile_ext]
except FileNotFoundError:
    print("femtoscope\data\mesh\geo\mesh_theta_int.geo cannot be found!")
    meshfile = "rec.vtk"
    # The same mesh is used for both the interior and the exterior regions
    MT = MeshingTools(dimension=2)
    MT.create_rectangle(dx=Rcut, dy=pi, centered=False)
    MT.create_subdomain(CellSizeMin=0.05, CellSizeMax=0.03, DistMax=0.0)
    MT.generate_mesh(meshfile, show_mesh=True, unique_boundary=False,
                     ignoreTags=[200, 202])
    meshfiles = [meshfile, meshfile]

def rho_func(coors):
    value = np.zeros(coors.shape[0])
    norm2 = coors[:,0]**2
    theta = coors[:, 1]
    boolin = np.where(norm2*((sin(theta)/R_ball)**2+(cos(theta)/R_ball)**2)<1)[0]
    value[boolin] = rho
    return value

densities_int = [rho_func]
densities_ext = None
pb_conn = PoissonSplit(alpha, densities_int, densities_ext, meshfiles,
                       coorsys=coorsys, conn=conn, fem_order=fem_order)
pb_conn.solve()
sol_conn = pb_conn.solver.sols[0]
field_conn = pb_conn.solver.weakforms[0].field
theta = arccos(Y_rand/rr)
coors_eval = np.ascontiguousarray(np.concatenate((rr[:, np.newaxis], theta[:, np.newaxis]), axis=1))
fem_conn = field_conn.evaluate_at(coors_eval, sol_conn[:, np.newaxis]).squeeze()
print("#DOFs = {}".format(sol_conn.shape[0]+pb_conn.solver.weakforms[1].field.coors.shape[0]))
print("mean pointwise relative error: {:.2E}".format(np.mean(abs((fem_conn-sol_ana)/sol_ana))))

        Solver:
            # weak-form(s): 2
            unbounded domain
            is linear?: YES

#DOFs = 17366
mean pointwise relative error: 1.05E-06


### (iii) Ping-pong

We can specify additional arguments when using the *ping-pong* technique:
- `relax_pp` is a relaxation parameter (the ping-pong technique is iterative)
- `max_iter_pp` is the maximum number of ping-pong iterations to be performed, if convergence has not been reached.

In [5]:
conn = 'ping-pong' # use the ping-pong technique

pb_pp = PoissonSplit(alpha, densities_int, densities_ext, meshfiles,
                     coorsys=coorsys, conn=conn, fem_order=fem_order,
                     relax_pp=0.7, max_iter_pp=10)
pb_pp.solve()
sol_pp = pb_pp.solver.sols[0]
field_pp = pb_pp.solver.weakforms[0].field
theta = arccos(Y_rand/rr)
coors_eval = np.ascontiguousarray(np.concatenate((rr[:, np.newaxis], theta[:, np.newaxis]), axis=1))
fem_pp = field_pp.evaluate_at(coors_eval, sol_pp[:, np.newaxis]).squeeze()
print("#DOFs = {}".format(sol_pp.shape[0]+pb_pp.solver.weakforms[1].field.coors.shape[0]))
print("mean pointwise relative error: {:.2E}".format(np.mean(abs((fem_pp-sol_ana)/sol_ana))))

        Solver:
            # weak-form(s): 2
            unbounded domain
            ping-pong max iter: 10
            is linear?: YES

ping-pong convergence in 8 iter
#DOFs = 17366
mean pointwise relative error: 4.00E-05


## Working example: Chameleon field on unbounded domain

This is the final example of this notebook, where we show how to compute the chameleon field on an unbounded domain, that is
$$
\begin{cases} 
  \alpha \Delta \phi = \rho - \phi^{-(n+1)} \text{ in } \Omega\\
  \phi(\mathbf{x}) \underset{\|\mathbf{x}\| \to +\infty}{\longrightarrow} 0
\end{cases} \, .
$$
We will use the 'virtual connection of DOFs' technique (`conn='connection'`) or the 'ping-pong' technique (`conn='ping-pong'`) in that specific case. The following code is taken from script `\script\chameleon_split.py`.

In [6]:
from femtoscope.physics.chameleon import ChameleonSplit
from femtoscope.inout.meshfactory import MeshingTools, mesh_from_geo
import numpy as np
from numpy import sqrt, pi, cos, sin, arccos

In [7]:
Rcut = 5.0 # radius for the separation of the two domains
sa = 1.0 # semi-major axis of the ellipsoid
sc = 1.0 # semi-minor axis of the ellipsoid
ecc = sqrt(1-(sc/sa)**2)
rho_bounds = [1e-1, 1e2] # = [rho_min, rho_max]
alpha = 1e-2
npot = 1
fem_order = 2
coorsys = 'polar' # 'cartesian' or 'polar' or 'polar_mu'
conn = 'connected' # 'connected' or 'ping-pong'
dimension = 2 # {1, 2, 3}
symmetry = True
update_mesh = True
analytic_params = {'R_A' : sa,
                   'rho_in' : max(rho_bounds),
                   'rho_vac' : min(rho_bounds)}
initial_guess = None # set to None or 'min_pot'
ent_func_in = []
ent_func_out = []

if coorsys == 'polar' and dimension == 2:
    meshfile_int = "mesh_theta_int.vtk"
    meshfile_ext = "mesh_theta_ext.vtk"
    try:
        assert sa == sc, "flat ellipsoid in cartesian coordinates only"
        size = 0.2
        mesh_from_geo('mesh_theta_int.geo', show_mesh=True,
                      param_dic={'size' : size, 'Ngamma' : int(1.5*pi/size),
                                 'Rc' : Rcut, 'sa' : sa, 'better_gamma' : 0})
        mesh_from_geo('mesh_theta_ext.geo', show_mesh=True,
                      param_dic={'size' : size, 'Ngamma' : int(1.5*pi/size),
                                 'Rc' : Rcut, 'better_gamma' : 0})
        meshfiles = [meshfile_int, meshfile_ext]
    except FileNotFoundError:
        print("femtoscope\data\mesh\geo\mesh_theta_int.geo cannot be found!")
        # The same mesh is used for both the interior and the exterior regions
        meshfiles = "rec.vtk"
        MT = MeshingTools(dimension=2)
        MT.create_rectangle(dx=Rcut, dy=pi, centered=False)
        MT.create_subdomain(CellSizeMin=0.05, CellSizeMax=0.05, DistMax=0.0)
        MT.generate_mesh(meshfiles, show_mesh=True, unique_boundary=False,
                         ignoreTags=[200, 202])

    def rho_func(coors):
        value = rho_bounds[0] * np.ones(coors.shape[0])
        norm2 = coors[:,0]**2
        theta = coors[:, 1]
        boolin = np.where(norm2*((sin(theta)/sa)**2 + (cos(theta)/sc)**2)<1)[0]
        value[boolin] = rho_bounds[1]
        return value

    densities_int = rho_func
    densities_ext = rho_bounds[0]
    
if coorsys == 'polar_mu' and dimension == 2:
    
    meshfile_int = "mesh_mu_int.vtk"
    meshfile_ext = "mesh_mu_ext.vtk"
    try:
        assert sa == sc, "flat ellipsoid in cartesian coordinates only"
        size = 0.1
        mesh_from_geo('mesh_mu_int.geo', show_mesh=True,
                      param_dic={'size' : size, 'Ngamma' : int(1.5*2/size),
                                 'Rc' : Rcut, 'sa' : sa, 'better_gamma' : 0})
        mesh_from_geo('mesh_mu_ext.geo', show_mesh=True,
                      param_dic={'size' : size, 'Ngamma' : int(1.5*2/size),
                                 'Rc' : Rcut, 'better_gamma' : 0})
        meshfiles = [meshfile_int, meshfile_ext]
    except FileNotFoundError:
        print("femtoscope\data\mesh\geo\mesh_mu_int.geo cannot be found!")
        meshfile = "rec.vtk"
        # The same mesh is used for both the interior and the exterior regions
        MT = MeshingTools(dimension=2)
        MT.create_rectangle(xll=0, yll=-1, dx=Rcut, dy=2, centered=False)
        MT.create_subdomain(CellSizeMin=0.5, CellSizeMax=0.5, DistMax=0.0)
        MT.generate_mesh(meshfile, show_mesh=True, unique_boundary=False,
                         ignoreTags=[200, 202])
        meshfiles = [meshfile, meshfile]
            
    def rho_func(coors):
        value = rho_bounds[0] * np.ones(coors.shape[0])
        norm2 = coors[:,0]**2
        mu = coors[:, 1]
        sin_theta = sqrt(1-mu**2)
        boolin = np.where(norm2*((sin_theta/sa)**2 + (mu/sc)**2)<1)[0]
        value[boolin] = rho_bounds[1]
        return value

    densities_int = rho_func
    densities_ext = rho_bounds[0]

elif coorsys == 'polar' and dimension == 1 and conn=='connected':
    '''Warning: the ping-pong technique does not work in 1D with the current
    version of femtoscope. It might be implemented in future versions'''
    from sfepy.examples.dg.example_dg_common import get_gen_1D_mesh_hook
    X1 = 0.0
    XN = Rcut
    n_nod = int(Rcut*2000) + 1
    n_el = n_nod - 1
    mesh = get_gen_1D_mesh_hook(X1, XN, n_nod).read(None)
    meshfiles = [mesh, mesh]

    def bord_zero(r, domain=None):
        return np.where(r==0)[0]

    def bord_Rcut(r, domain=None):
        return np.where(r==Rcut)[0]

    def rho_func(r):
        return np.where(r<=sa, rho_bounds[1], rho_bounds[0])

    densities_int = rho_func
    densities_ext = rho_bounds[0]
    ent_func_in = [(0, bord_Rcut)]
    ent_func_out = [(0, bord_zero), (0, bord_Rcut)]

elif coorsys == 'cartesian' and dimension == 2:
    # Meshes creation
    Ngamma = 300 # Number of vertices on the shared boundary

    '''
    The interior mesh and the exterior mesh are different:
        - the interior mesh contains a subdomain representing the main body
        - the exterior mesh represents vacum extending from Rcut up to
        infinity. Thus, it must contain a node at (0,0,0) so as to impose a
        Dirichlet boundary condition at infinity.
    In order to make it possible for the two meshes to exchange information
    at their common boundary, one must ensure that the two meshes match
    perfectly at the interface.
    '''

    meshfile1 = 'cir_in.vtk'
    meshfile2 = 'cir_out.vtk'
    MT = MeshingTools(dimension=2)
    s1 = MT.create_ellipse(sa, sc)
    s = s1
    MT.create_subdomain(CellSizeMin=0.05, CellSizeMax=0.2, DistMin=sa/5,
                        DistMax=3*sa)
    s2 = MT.create_disk_from_pts(Rcut, N=Ngamma) # impose vertices on gamma
    s12 = MT.subtract_shapes(s2, s, removeObject=True, removeTool=False)
    MT.create_subdomain(CellSizeMin=0.15, CellSizeMax=0.2, DistMax=3)
    MT.generate_mesh(meshfile1, show_mesh=True, symmetry=symmetry,
                     ignoreTags=[200])

    MT = MeshingTools(dimension=2)
    MT.create_disk_from_pts(Rcut, N=Ngamma) # impose vertices on gamma
    MT.create_subdomain(CellSizeMin=0.15, CellSizeMax=0.4, DistMin=0.0,
                      DistMax=2)
    center_rf = [0.05, 0.15, 0.1, 3]
    MT.generate_mesh(meshfile2, show_mesh=True, embed_center=True,
                     symmetry=symmetry, center_rf=center_rf)

    densities_int = [rho_bounds[1], rho_bounds[0]]
    densities_ext = rho_bounds[0]
    meshfiles = [meshfile1, meshfile2]

elif coorsys == 'cartesian' and dimension == 3:
    densities_int = [rho_bounds[1], rho_bounds[0]]
    densities_ext = rho_bounds[0]
    meshfile = 'sphere.vtk'
    center_rf = [1, 1, 1, 1]
    MT = MeshingTools(dimension=3)
    s1 = MT.create_ellipsoid(sa, sa, sa)
    MT.create_subdomain(CellSizeMin=0.7, CellSizeMax=0.7)
    s2 = MT.create_ellipsoid(5, 5, 5)
    s12 = MT.subtract_shapes(s2, s1, removeObject=True, removeTool=False)
    MT.create_subdomain(CellSizeMin=0.7, CellSizeMax=0.7)
    meshfiles = MT.generate_mesh(meshfile, show_mesh=True, convert=True,
                                 exterior=True, center_rf=center_rf, ignoreTags=[200])
    
pb = ChameleonSplit(alpha, npot, densities_int, densities_ext, rho_bounds, meshfiles, coorsys=coorsys,
                    conn=conn, fem_order=fem_order, min_iter=5, max_iter=200, analytic_params=analytic_params,
                    relax=0.7, initial_guess=initial_guess, Rcut=Rcut, entity_functions_in=ent_func_in,
                    entity_functions_out=ent_func_out)
pb.solve(save_all_newton=False)

        Solver:
            # weak-form(s): 2
            unbounded domain
            is linear?: NO
            relax parameter: 0.7
        Stop Criteria:
            max iter: 200
            relative delta_sol tolerance: 1e-14
            relative delta_res tolerance: 1e-14

************ Iteration no 0 ************
Sol rel delta     NAN
Residual L2       5.589E+00
Residual mean     3.094E-02
Residual inf      9.740E-01
Residual rel delta NAN

************ Iteration no 1 ************
Sol rel delta     1.797E-01
Residual L2       1.132E+00
Residual mean     9.353E-03
Residual inf      2.115E-01
Residual rel delta 1.042E+00

************ Iteration no 2 ************
Sol rel delta     3.309E-02
Residual L2       5.909E-01
Residual mean     5.186E-03
Residual inf      9.207E-02
Residual rel delta 4.961E-01

************ Iteration no 3 ************
Sol rel delta     3.896E-02
Residual L2       3.193E-01
Residual mean     2.874E-03
Residual inf      4.840E-02
Residual rel delta 4.926E-01


Some remarks:
- The function `pb.solve` can take the keyword argument `save_all_newton`. When set to `True`, this will create a new directory in `femtoscope\data\result\` containing all the iterates of Newton's method (solution, residual vector, ...) for any thorough post-checking of the data.
- The residual vector is computed only in the interior domain $\Omega_{\mathrm{int}}$ for technical reasons (it can only be computed on bounded domains).
- The norm-2 of the residual vector might stagnate before the iterations have fully converged. This can be due to fact that the residual remains *big* in some localized region of the mesh while it still decreases in regions where it is already *smaller*. One must be wary of this single number which is not always representative of the full picture.